In [61]:
import numpy as np
import pickle
import tensorflow as tf
from googletrans import Translator
translator = Translator()
from tensorflow.keras.models import load_model
model = load_model('./model/chatbot_model.h5')
import json
import random
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopWords = stopwords.words('english')

tags = pickle.load(open('./model/tags.pkl','rb'))
patterns = pickle.load(open('./model/patterns.pkl','rb'))
responses = pickle.load(open('./model/responses.pkl','rb'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
def splitDataset(inputs, labels):
    totalData = len(inputs)
    valSize = int(totalData * 0.2)
    valSplit = list(range(0, valSize * 5, 5))
    trainSplit = [i for i in range(totalData) if i not in valSplit]


    trainTexts = [inputs[i] for i in trainSplit]
    valTexts = [inputs[i] for i in valSplit]
    trainLabels = [labels[i] for i in trainSplit]
    valLabels = [labels[i] for i in valSplit]

    trainDataset = tf.data.Dataset.from_tensor_slices((trainTexts, trainLabels))
    valDataset = tf.data.Dataset.from_tensor_slices((valTexts, valLabels))

    return trainDataset, valDataset
    
trainDataset, valDataset = splitDataset(patterns, tags)

In [63]:
def standardizeFunc(sentence):
    stopwords = stopWords
    sentence = tf.strings.lower(sentence)

    for word in stopwords:
        if word[0] == "'":
            sentence = tf.strings.regex_replace(sentence, rf"{word}\b", "")
        else:
            sentence = tf.strings.regex_replace(sentence, rf"\b{word}\b", "")
    sentence = tf.strings.regex_replace(sentence, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', "")

    return sentence

In [64]:
maxLeght = 30
def fitVectorizer(trainSentences, standardizeFunc):
    vectorizer = tf.keras.layers.TextVectorization(
		standardize=standardizeFunc,
    output_sequence_length=maxLeght
	)
    vectorizer.adapt(trainSentences)
    return vectorizer

textDataset = trainDataset.map(lambda text, label: text)
vectorizer = fitVectorizer(textDataset, standardizeFunc)

In [65]:
def fitLabelEncoder(trainLabel, valLabel):
    trainLabel = list(trainLabel.as_numpy_iterator())
    valLabel = list(valLabel.as_numpy_iterator())
    labels = trainLabel + valLabel

    labelEncoder = tf.keras.layers.StringLookup(num_oov_indices=0)
    labelEncoder.adapt(labels)

    return labelEncoder

trainLabels = trainDataset.map(lambda text, label: label)
valLabels = valDataset.map(lambda text, label: label)
labelEncoder = fitLabelEncoder(trainLabels,valLabels)

In [72]:
def getResponse(predictTag, responses, confidence):
    if confidence < 0.6:
        return "Sorry, the input you entered is not clear."
    return np.random.choice(responses[predictTag])


def showRespon(input):
    inputA = vectorizer([input])
    prediction = model.predict(inputA)

    predictClassIndex = np.argmax(prediction, axis=-1)
    confidence = prediction[0][predictClassIndex[0]]
    predictTag = labelEncoder.get_vocabulary()[predictClassIndex[0]]
    response = getResponse(predictTag, responses, confidence)

    return predictTag, response, confidence

def chatbot():
    inputUser = translator.translate(input("User: "), src="id", dest="en")
    testInput = inputUser.text
    predictTag, response, confidence = showRespon(testInput)
    print(f"Prediksi Masalah : ", translator.translate(predictTag, src="en", dest="id").text)
    print(f"PyChan: ", translator.translate(response, src="en", dest="id").text)

while True:
    chatbot()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediksi Masalah :  salam
PyChan:  Hai, apakah ada yang bisa saya bantu?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediksi Masalah :  Demam
PyChan:  Untuk mengobati demam di rumah: 1) Minum banyak cairan agar tetap terhidrasi.2) Berpakaian dengan pakaian ringan.3) Gunakan selimut ringan jika Anda merasa dingin, sampai kedinginan berakhir.4) Ambil acetaminophen (Tylenol, lainnya) atau ibuprofen (Advil, Motrin Ib, lainnya).5) Dapatkan bantuan medis jika demam berlangsung lebih dari lima hari berturut -turut.


TypeError: the JSON object must be str, bytes or bytearray, not NoneType